In [2]:
import torch
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os


if not os.path.exists('./vae_img'):
    os.mkdir('./vae_img')


def to_img(x):
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor()
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE()
if torch.cuda.is_available():
    model.cuda()

reconstruction_function = nn.BCELoss(size_average=False)


def loss_function(recon_x, x, mu, logvar):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    BCE = reconstruction_function(recon_x, x)  # mse loss
    # loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    # KL divergence
    return BCE + KLD


optimizer = optim.Adam(model.parameters(), lr=1e-3)


for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(dataloader):
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img)
        if torch.cuda.is_available():
            img = img.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(img)
        loss = loss_function(recon_batch, img, mu, logvar)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(img), len(dataloader.dataset),
                100. * batch_idx / len(dataloader),
                loss.data[0] / len(img)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(dataloader.dataset)))
    if epoch % 10 == 0:
        save = to_img(recon_batch.cpu().data)
        save_image(save, './vae_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './vae.pth')

Processing...
Done!
Train Epoch: 0 [0/60000 (0%)]	Loss: 549.602173
Train Epoch: 0 [12800/60000 (21%)]	Loss: 178.651962
Train Epoch: 0 [25600/60000 (43%)]	Loss: 148.502716
Train Epoch: 0 [38400/60000 (64%)]	Loss: 136.507996
Train Epoch: 0 [51200/60000 (85%)]	Loss: 131.207855
====> Epoch: 0 Average loss: 164.7680
Train Epoch: 1 [0/60000 (0%)]	Loss: 128.002197
Train Epoch: 1 [12800/60000 (21%)]	Loss: 124.491119
Train Epoch: 1 [25600/60000 (43%)]	Loss: 122.570290
Train Epoch: 1 [38400/60000 (64%)]	Loss: 120.448181
Train Epoch: 1 [51200/60000 (85%)]	Loss: 118.580719
====> Epoch: 1 Average loss: 121.6633
Train Epoch: 2 [0/60000 (0%)]	Loss: 116.891281
Train Epoch: 2 [12800/60000 (21%)]	Loss: 112.851990
Train Epoch: 2 [25600/60000 (43%)]	Loss: 116.979782
Train Epoch: 2 [38400/60000 (64%)]	Loss: 108.762390
Train Epoch: 2 [51200/60000 (85%)]	Loss: 115.179276
====> Epoch: 2 Average loss: 114.7248
Train Epoch: 3 [0/60000 (0%)]	Loss: 109.678757
Train Epoch: 3 [12800/60000 (21%)]	Loss: 109.772255
Tr

Train Epoch: 26 [51200/60000 (85%)]	Loss: 103.826279
====> Epoch: 26 Average loss: 103.1749
Train Epoch: 27 [0/60000 (0%)]	Loss: 102.304779
Train Epoch: 27 [12800/60000 (21%)]	Loss: 100.821182
Train Epoch: 27 [25600/60000 (43%)]	Loss: 101.349503
Train Epoch: 27 [38400/60000 (64%)]	Loss: 106.669678
Train Epoch: 27 [51200/60000 (85%)]	Loss: 101.252121
====> Epoch: 27 Average loss: 103.0511
Train Epoch: 28 [0/60000 (0%)]	Loss: 98.573929
Train Epoch: 28 [12800/60000 (21%)]	Loss: 104.944794
Train Epoch: 28 [25600/60000 (43%)]	Loss: 102.061768
Train Epoch: 28 [38400/60000 (64%)]	Loss: 101.531120
Train Epoch: 28 [51200/60000 (85%)]	Loss: 104.182983
====> Epoch: 28 Average loss: 102.9653
Train Epoch: 29 [0/60000 (0%)]	Loss: 101.642212
Train Epoch: 29 [12800/60000 (21%)]	Loss: 100.676529
Train Epoch: 29 [25600/60000 (43%)]	Loss: 106.681923
Train Epoch: 29 [38400/60000 (64%)]	Loss: 102.400009
Train Epoch: 29 [51200/60000 (85%)]	Loss: 105.030701
====> Epoch: 29 Average loss: 102.9002
Train Epoch:

Train Epoch: 54 [25600/60000 (43%)]	Loss: 103.500137
Train Epoch: 54 [38400/60000 (64%)]	Loss: 104.473938
Train Epoch: 54 [51200/60000 (85%)]	Loss: 103.281937
====> Epoch: 54 Average loss: 101.6248
Train Epoch: 55 [0/60000 (0%)]	Loss: 101.759010
Train Epoch: 55 [12800/60000 (21%)]	Loss: 103.535652
Train Epoch: 55 [25600/60000 (43%)]	Loss: 98.822021
Train Epoch: 55 [38400/60000 (64%)]	Loss: 100.335503
Train Epoch: 55 [51200/60000 (85%)]	Loss: 102.087418
====> Epoch: 55 Average loss: 101.5375
Train Epoch: 56 [0/60000 (0%)]	Loss: 102.197433
Train Epoch: 56 [12800/60000 (21%)]	Loss: 102.646561
Train Epoch: 56 [25600/60000 (43%)]	Loss: 102.559830
Train Epoch: 56 [38400/60000 (64%)]	Loss: 100.470871
Train Epoch: 56 [51200/60000 (85%)]	Loss: 100.216576
====> Epoch: 56 Average loss: 101.5349
Train Epoch: 57 [0/60000 (0%)]	Loss: 101.261673
Train Epoch: 57 [12800/60000 (21%)]	Loss: 104.561546
Train Epoch: 57 [25600/60000 (43%)]	Loss: 103.719345
Train Epoch: 57 [38400/60000 (64%)]	Loss: 101.49749

====> Epoch: 81 Average loss: 100.8985
Train Epoch: 82 [0/60000 (0%)]	Loss: 101.094467
Train Epoch: 82 [12800/60000 (21%)]	Loss: 101.404732
Train Epoch: 82 [25600/60000 (43%)]	Loss: 101.423439
Train Epoch: 82 [38400/60000 (64%)]	Loss: 98.304504
Train Epoch: 82 [51200/60000 (85%)]	Loss: 96.529991
====> Epoch: 82 Average loss: 100.8761
Train Epoch: 83 [0/60000 (0%)]	Loss: 101.400757
Train Epoch: 83 [12800/60000 (21%)]	Loss: 101.524460
Train Epoch: 83 [25600/60000 (43%)]	Loss: 100.132683
Train Epoch: 83 [38400/60000 (64%)]	Loss: 97.242775
Train Epoch: 83 [51200/60000 (85%)]	Loss: 100.490479
====> Epoch: 83 Average loss: 100.8592
Train Epoch: 84 [0/60000 (0%)]	Loss: 102.775070
Train Epoch: 84 [12800/60000 (21%)]	Loss: 99.997490
Train Epoch: 84 [25600/60000 (43%)]	Loss: 96.824860
Train Epoch: 84 [38400/60000 (64%)]	Loss: 100.991318
Train Epoch: 84 [51200/60000 (85%)]	Loss: 102.006088
====> Epoch: 84 Average loss: 100.8346
Train Epoch: 85 [0/60000 (0%)]	Loss: 102.845795
Train Epoch: 85 [1280